In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
import pandas as pd
import numpy as np
import datetime
!pip install pandas_summary
from pandas_summary import DataFrameSummary
from lightgbm import LGBMRegressor
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
Path="/content/gdrive/MyDrive/Colab Notebooks/kaggle-rossmann-master Limpio/"


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df = pd.read_feather(Path+'test_normalized_data_withoutOutliers.fth')
df_test = pd.read_feather(Path+'test_normalized_data.fth')

In [ ]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

In [ ]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Precipitationmm',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday', 'StateHoliday_bool']   

In [ ]:
y_out_columns = ['Sales']

In [ ]:

df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

Cantidad en val: 30188, porcentaje: 0.9642465458145908


In [ ]:
X_train_full=df[cat_vars + contin_vars]
X_train = df_train[cat_vars + contin_vars]
X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [ ]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    y_train_full=np.log(df[y_out_columns].values)/max_log_y
    y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    y_train_full = (dfsinout[y_out_columns].values - y_mean)/y_std
    y_val = (df_val[y_out_columns].values - y_mean)/y_std

In [ ]:
cl4=  {'colsample_bytree': 0.5,
    'learning_rate': 0.05,
    'min_child_samples': 200}
cl1=  {'max_depth': 400,
    'learning_rate': 0.01}
cl_opt={'colsample_bytree': 0.5192643198735548,
  'learning_rate': 0.05161157303322725,
  'max_depth': 500,
  'min_child_samples': 194,
  'min_child_weight': 0,
  'num_leaves': 53,
  'reg_alpha': 1.0,
  'reg_lambda': 1.5130794435785242e-07}

# Nueva sección

In [ ]:
RS=False
if RS==True:
  n_estimators=2500
  cv_s = [(np.arange(30188), np.arange(30188, 814150))]
  model = LGBMRegressor(n_estimators=n_estimators)
  param_dist = {"max_depth": np.linspace(10, 600,100,dtype=int),              
              'learning_rate': np.linspace(0.01, 0.2,100),
              'reg_lambda': np.logspace(-9, -3, 100),
              'min_child_samples': np.linspace(1, 400,dtype=int),
              'min_child_weight': np.logspace(-8, -1, 100),
              'num_leaves': randint(5, 80),
              'colsample_bytree': np.linspace(0.3, 0.95,100),
              'reg_alpha': np.linspace(0, 1,20)
              }
 
  clf = RandomizedSearchCV(model, param_dist, n_jobs=-1, verbose=1, cv=cv_s, n_iter=500)
  
  fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'rmse', 
            "eval_set" : [(X_val, y_val.reshape(-1))],
            'eval_names': ['valid'],
            'verbose': 10,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars
           }
  clf.fit(X_train, y_train.reshape(-1), **fit_params)
  print(clf.best_params_)
  results_df = pd.DataFrame(clf.cv_results_)
  results_df.drop(columns=['std_fit_time', 'mean_score_time', 'std_score_time', 'params', 
                         'std_test_score']).sort_values('rank_test_score').head(10)
  print(results_df)
  from sklearn.externals import joblib
  
  joblib.dump(clf.best_params_, 'bestparm.pkl')
  joblib.dump(clf.cv_results_, "cvresult.pkl")

In [ ]:
FullRun=False
if FullRun==True:
  fit_params={'verbose': 10,
            'categorical_feature': cat_vars
           }
  n_estimators=2500
  model = LGBMRegressor(n_estimators=n_estimators, **cl4)
  model.fit(X_train_full, y_train_full.reshape(-1), **fit_params)
  if log_output:
    y_pred_train = np.exp(model.predict(X_train_full, verbose=1)*max_log_y)
    y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
  else:
    y_pred_train = model.predict(X_train_full, verbose=1)*y_std + y_mean
    y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
    y_pred_test = model.predict(X_test, verbose=1)*y_std +  y_mean
  #rmseTrain=np.sqrt((((df['Sales'].values - y_pred_train)/df['Sales'].values)**2).sum()/len(y_pred_train))
  #rmseVal=np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))
  #print("RMSE Train"+rmseTrain)
  #print("RMSE Val"+rmseVal)


In [ ]:
UniqueRun=True
if UniqueRun==True:
  fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'rmse', 
            "eval_set" : [(X_val, y_val.reshape(-1))],
            'eval_names': ['valid'],
            'verbose': 10,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars
           }
  n_estimators=2500
  model = LGBMRegressor(n_estimators=n_estimators, **cl_opt)
  model.fit(X_train, y_train.reshape(-1), **fit_params)


  if log_output:
    y_pred_train = np.exp(model.predict(X_train, verbose=1)*max_log_y)
    y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
  else:
    y_pred_train = model.predict(X_train, verbose=1)*y_std + y_mean
    y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
    y_pred_test = model.predict(X_test, verbose=1)*y_std +  y_mean
  #rmseTrain=np.sqrt((((df['Sales'].values - y_pred_train)/df['Sales'].values)**2).sum()/len(y_pred_train))
  #rmseVal=np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))
  #print("RMSE Train"+rmseTrain)
  #print("RMSE Val"+rmseVal)


/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Assortment', 'CompetitionMonthsOpen', 'CompetitionOpenSinceYear', 'Day', 'DayOfWeek', 'Events', 'Month', 'Promo2SinceYear', 'Promo2Weeks', 'PromoInterval', 'Promo_bw', 'Promo_fw', 'SchoolHoliday_bw', 'SchoolHoliday_fw', 'State', 'StateHoliday', 'StateHoliday_bool_bw', 'StateHoliday_bool_fw', 'Store', 'StoreType', 'Week', 'Year']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 100 rounds.
[10]	valid's l2: 0.000933025	valid's rmse: 0.0305455
[20]	valid's l2: 0.000646345	valid's rmse: 0.0254233
[30]	valid's l2: 0.000502034	valid's rmse: 0.0224061
[40]	valid's l2: 0.000359304	valid's rmse: 0.0189553
[50]	valid's l2: 0.000302682	valid's rmse: 0.0173978
[60]	valid's l2: 0.000267547	valid's rmse: 0.0163569
[70]	valid's l2: 0.000242076	valid's rmse: 0.0155588
[80]	valid's l2: 0.000225943	valid's rmse: 0.0150314
[90]	valid's l2: 0.000214196	valid's rmse: 0.0146354
[100]	valid's l2: 0.000202456	valid's rmse: 0.0142287
[110]	valid's l2: 0.000196152	valid's rmse: 0.0140054
[120]	valid's l2: 0.00018932	valid's rmse: 0.0137594
[130]	valid's l2: 0.000183903	valid's rmse: 0.0135611
[140]	valid's l2: 0.000177766	valid's rmse: 0.0133329
[150]	valid's l2: 0.000171407	valid's rmse: 0.0130923
[160]	valid's l2: 0.000167911	valid's rmse: 0.012958
[170]	valid's l2: 0.000163807	valid's rmse: 0.0127987
[180]	valid's l2: 0.000159377	

# Sumbit a la competición

In [ ]:
sample_csv = pd.read_csv(Path+'sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'{Path}submision2LIGHTGBMsinOutliers19M_1.csv', index=False)